In [2]:
#import libraries
#!pip install kaggle
import kaggle

In [3]:
!kaggle datasets download ankitbansal06/retail-orders -f orders.csv


  0%|          | 0.00/200k [00:00<?, ?B/s]
100%|██████████| 200k/200k [00:00<00:00, 1.67MB/s]
100%|██████████| 200k/200k [00:00<00:00, 1.67MB/s]


Dataset URL: https://www.kaggle.com/datasets/ankitbansal06/retail-orders
License(s): CC0-1.0



In [5]:
#extract file from zip file
import zipfile
zip_ref = zipfile.ZipFile('orders.csv.zip')
zip_ref.extractall()
zip_ref.close()

In [9]:
#read data from the file and handle null values
import pandas as pd
df = pd.read_csv('orders.csv',na_values=['Not Available', 'unknown'])
#df.head(20)
df['Ship Mode'].unique()

array(['Second Class', 'Standard Class', nan, 'First Class', 'Same Day'],
      dtype=object)

In [14]:
#rename columns names ..make them lower case and replace space with underscore
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(" ","_")
df.head(5)

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5


In [20]:
#Creating new columns discount, sale price and profit
df['discount']=df['list_price'] * df['discount_percent']/100
df['sale_price'] = df['list_price']-df['discount']
df['profit']=df['sale_price']-df['cost_price']
df.head(4)


,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent,discount,sale_price,profit
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2,5.2,254.8,14.8
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3,21.9,708.1,108.1
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5,0.5,9.5,-0.5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2,19.2,940.8,160.8


In [32]:
#convert order date from object data type to datetime
df.dtypes
df['order_date'] = pd.to_datetime(df['order_date'], format="%Y-%m-%d")

print(df.dtypes)
print(df.head())


order_id                     int64
order_date          datetime64[ns]
ship_mode                   object
segment                     object
country                     object
city                        object
state                       object
postal_code                  int64
region                      object
category                    object
sub_category                object
product_id                  object
cost_price                   int64
list_price                   int64
quantity                     int64
discount_percent             int64
discount                   float64
sale_price                 float64
profit                     float64
dtype: object
   order_id order_date       ship_mode    segment        country  \
0         1 2023-03-01    Second Class   Consumer  United States   
1         2 2023-08-15    Second Class   Consumer  United States   
2         3 2023-01-10    Second Class  Corporate  United States   
3         4 2022-06-18  Standard Class   Consumer

In [37]:
#drop cost price list price and discount percent columns
df.drop(columns=['list_price','cost_price','discount_percent'], inplace=True)
df

KeyError: "['list_price', 'cost_price', 'discount_percent'] not found in axis"

In [47]:
#load the data into sql server using replace option
!pip install pyodbc
import sqlalchemy as sal
engine = sal.create_engine('mssql://Praneeth-09/master?driver=ODBC+DRIVER+17+FOR+SQL+SERVER')
conn = engine.connect()

In [48]:
df.to_sql('df_orders', con=conn, index=False)

38